In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import timeit

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_score, recall_score, accuracy_score, auc
from hmeasure import h_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, precision_recall_fscore_support,roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras import initializers
from keras.models import Sequential
from keras.optimizers import Adam

# Read the data using csv
path_train = r'C:\Users\...........\credit_risk_dataset_cleanedtraining.csv'
df_train = pd.read_csv(path_train, encoding = "ISO-8859-1")

path_test = r'C:\Users\............\credit_risk_dataset_cleanedtesting.csv'
df_test = pd.read_csv(path_test, encoding = "ISO-8859-1")

# Prin 5 forst lines of df
df_train.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership,loan_intent,loan_grade,cb_person_default_on_file,loan_status
0,23,120000,1.0,25600,12.69,0.21,3,3,1,1,0,0
1,21,42500,3.0,25000,12.73,0.59,3,3,1,2,1,1
2,26,48000,2.0,25000,9.64,0.52,4,3,1,1,0,1
3,25,42360,1.0,25000,16.35,0.59,3,3,1,4,1,1
4,23,50000,1.0,25000,7.90,0.50,3,3,0,0,0,1


In [5]:
#pip install hmeasure

In [2]:
X_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]

X_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)

In [3]:
class tuning_methods:
    
    def __init__(self, X_train, X_val, y_train,  y_val, X_test, y_test):
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        
    
    # Define the Keras model
    def create_model(self, units, alpha, dropout, l2_kern, l2_bias, lr, batch_size):        # 
        
        model = Sequential()
        model.add(Dense(units, input_shape=(self.X_train.shape[1],), 
                                 activation =model.add(LeakyReLU(alpha=alpha)),
                                 kernel_regularizer = l2(l2_kern),
                                 bias_regularizer = l2(l2_bias),
                                 kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05)
            ))
        model.add(Dropout(dropout))
        model.add(Dense(1, activation='sigmoid'))
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['AUC'])
        return model
    
    
    # Define the Optuna study
    def objective(self, trial):
        
        # Define the pipeline
        pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
                ])

        # Set hyperparameters
        units = trial.suggest_int('units', 2, 40)
        alpha = trial.suggest_float('alpha', 0.01, 0.1)
        dropout = trial.suggest_float("dropout", 0.2, 0.5)    
        lr = trial.suggest_float("lr", 5e-5, 1e-2, log=True)
        batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
        l2_kern = trial.suggest_float("l2_kern",  1e-5, 1e-2, log=True)
        l2_bias = trial.suggest_float("l2_bias",  1e-5, 1e-2, log=True)
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        pipeline.set_params(classifier__units=units,
                        classifier__alpha=alpha,
                        classifier__l2_kern=l2_kern,
                        classifier__l2_bias=l2_bias,
                        classifier__dropout=dropout,
                        classifier__lr=lr,
                        classifier__epochs = epochs,
                        classifier__batch_size = batch_size,
                        classifier__validation_data = validation_data,
                        classifier__callbacks=[early_stopping])
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2023)
        scores = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
    
        if trial.should_prune():
            raise optuna.TrialPruned()

        return np.mean(scores)

    def Tuning(self, n_trials=100, sampler = optuna.samplers.TPESampler()):
        
        start = timeit.default_timer()

        study = optuna.create_study(direction="maximize",             
                            sampler=sampler, 
                            pruner= optuna.pruners.MedianPruner()
                           )
        study.optimize(self.objective, n_trials=n_trials)  # to be converged it needs atleast 200 trials 

        print("Number of finished trials: {}".format(len(study.trials)))

        print("Best trial:")
        trial = study.best_trial

        print("  Value: {}".format(trial.value))

        stop = timeit.default_timer()

        time_opt = stop - start
        params_TPE = []

        for key, value in trial.params.items():
            params_TPE.append([key,value])
            print("    {}: {}".format(key, value))
            
        params_TPE.append(["time",round(time_opt/60,2)])

        return params_TPE
    
    def Best_MLP_model(self, params):
        units = params[0][1]
        alpha = np.round(params[1][1],3)
        dropout = np.round(params[2][1],2)
        lr = np.round(params[3][1],5)
        batchs = params[4][1]
        kernel_regularizer=np.round(params[5][1],5)
        bias_regularizer=np.round(params[6][1],5)

        # Define the pipeline
        pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))
                ])

        # Set hyperparameters
        epochs = 200
        validation_data = (self.X_val, self.y_val)
        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        pipeline.set_params(classifier__units=units,
                        classifier__alpha=alpha,
                        classifier__l2_kern=kernel_regularizer,
                        classifier__l2_bias=bias_regularizer,
                        classifier__dropout=dropout,
                        classifier__lr=lr,
                        classifier__epochs = epochs,
                        classifier__batch_size = batchs,
                        classifier__validation_data = validation_data,
                        classifier__callbacks=[early_stopping])
        
        # Use cross validation to monitor the deferrence between training and testing data as an evidence of overfitting
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_train = cross_val_score(pipeline, self.X_train, self.y_train, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_train = np.mean(scores_train)

        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)
        scores_test = cross_val_score(pipeline, self.X_test, self.y_test, cv=kfold, scoring="roc_auc", error_score='raise')
        mean_score_test = np.mean(scores_test)

        print(f"Train mean AUC score is {mean_score_train} while test mean AUC score is {mean_score_test}.")
      
        # Fit the pipeline to the training data
        history = pipeline.fit(self.X_train, self.y_train)

        #score = pipeline.evaluate(X_test, y_test, verbose=0)  

        y_pred_prob_tpe = pipeline.predict(self.X_test)

        y_pred_tpe = y_pred_prob_tpe.round()

        print(classification_report(self.y_test, y_pred_tpe, target_names = ['No','Yes']))

        acc = accuracy_score(self.y_test, y_pred_tpe)
        pre = precision_score(self.y_test, y_pred_tpe)
        rec = recall_score(self.y_test, y_pred_tpe)
        f1 = f1_score(self.y_test, y_pred_tpe)
        auc = roc_auc_score(self.y_test, y_pred_tpe)
        y_test_array = np.array(self.y_test)
        y_pred_tpe_array = np.array(y_pred_tpe)
        H_measure = h_score(y_test_array, y_pred_tpe_array)

        opt = [acc, pre, rec, f1, auc, H_measure]

        scores = pd.DataFrame(opt, index =['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', "H-measure"], 
                              columns =['Tree-structured Parzen Estimator'])
        
        return scores 

## Take scores for every tuning algorithm

In [ ]:
tun = tuning_methods(X_train, X_val, y_train,  y_val, X_test, y_test)
trials = 100

# Tree-structured Parzen Estimator
params_tpe = tun.Tuning(n_trials=trials, sampler = optuna.samplers.TPESampler())
tpe_scores = tun.Best_MLP_model(params_tpe)

[I 2023-03-01 21:31:12,282] A new study created in memory with name: no-name-a40475db-4f9d-4596-b00f-48afa465a8a1
C:\Users\User\AppData\Local\Temp\ipykernel_9208\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-01 21:32:28,156] Trial 0 finished with value: 0.8535131674171074 and parameters: {'units': 34, 'alpha': 0.09822043197323257, 'dropout': 0.21817536378332675, 'lr': 0.0006970271511110475, 'batch_size': 32, 'l2_kern': 0.0031146406105100775, 'l2_bias': 1.5730042323055288e-05}. Best is trial 0 with value: 0.8535131674171074.
C:\Users\User\AppData\Local\Temp\ipykernel_9208\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-01 21:33:48,888] Trial 1 finished with value: 0.8495650501409896 and parameters: {'units': 3, 'alpha': 0.013845901865392926, 'dropout': 0.4933531551725452, 'lr': 0.0071989471295323595, 'batch_size': 16, 'l2_kern': 0.006963743187804702, 'l2_bias': 0.0011863842767857447}. Best is trial 0 with value: 0.8535131674171074.
C:\Users\User\AppData\Local\Temp\ipykernel_9208\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

[I 2023-03-01 22:57:50,951] Trial 6 finished with value: 0.8531465374208745 and parameters: {'units': 35, 'alpha': 0.04948246889680272, 'dropout': 0.3803869248313516, 'lr': 0.001565171085415664, 'batch_size': 16, 'l2_kern': 0.000500200182330259, 'l2_bias': 6.643061421299829e-05}. Best is trial 5 with value: 0.8545862462489718.
C:\Users\User\AppData\Local\Temp\ipykernel_9208\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-01 22:58:23,952] Trial 7 finished with value: 0.8540396685385495 and parameters: {'units': 24, 'alpha': 0.05639199287957771, 'dropout': 0.4907007096866018, 'lr': 0.00033155643223441807, 'batch_size': 64, 'l2_kern': 0.002349108143115436, 'l2_bias': 0.0006020787933325963}. Best is trial 5 with value: 0.8545862462489718.
C:\Users\User\AppData\Local\Temp\ipykernel_9208\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 1ms/step


[I 2023-03-01 22:59:56,215] Trial 8 finished with value: 0.8536719703170624 and parameters: {'units': 28, 'alpha': 0.09786803463698197, 'dropout': 0.299241243248711, 'lr': 0.0009335772575069589, 'batch_size': 16, 'l2_kern': 0.0004467639299913888, 'l2_bias': 0.006597077807933503}. Best is trial 5 with value: 0.8545862462489718.


In [4]:
# Random Sampler
params_rs = tun.Tuning(n_trials=trials, sampler = optuna.samplers.RandomSampler())
rs_scores = tun.Best_MLP_model(params_rs)
rs_scores = rs_scores.rename(columns={'Tree-structured Parzen Estimator': 'Random Sampler'})

[I 2023-03-02 06:48:11,956] A new study created in memory with name: no-name-006a02e6-a616-4c1d-9de1-4c276fd4e26c
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 06:49:44,061] Trial 0 finished with value: 0.8543616267603878 and parameters: {'units': 29, 'alpha': 0.0933619000305146, 'dropout': 0.4227613860657616, 'lr': 6.425006258745942e-05, 'batch_size': 256, 'l2_kern': 4.762110802438709e-05, 'l2_bias': 2.350471811023897e-05}. Best is trial 0 with value: 0.8543616267603878.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 1s 3ms/step


[I 2023-03-02 06:50:30,173] Trial 1 finished with value: 0.8519218039736866 and parameters: {'units': 33, 'alpha': 0.06955928315319992, 'dropout': 0.4465319406693177, 'lr': 0.004565620660087131, 'batch_size': 64, 'l2_kern': 0.006847973069366916, 'l2_bias': 0.001342025956847735}. Best is trial 0 with value: 0.8543616267603878.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 08:08:35,718] Trial 99 finished with value: 0.8551934128229037 and parameters: {'units': 5, 'alpha': 0.05668036856331818, 'dropout': 0.3458406584357721, 'lr': 0.00015979630519304404, 'batch_size': 32, 'l2_kern': 0.00038178605940627944, 'l2_bias': 1.840931921076541e-05}. Best is trial 35 with value: 0.8555828655755575.


Number of finished trials: 100
Best trial:
  Value: 0.8555828655755575
    units: 13
    alpha: 0.026858226309700473
    dropout: 0.27767830829114143
    lr: 0.0002811086545566117
    batch_size: 256
    l2_kern: 0.0008024195551368709
    l2_bias: 0.0007774044467709288


C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:111: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8551909266930217 while test mean AUC score is 0.8598668744838935.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.88      0.96      0.92      4648
         Yes       0.76      0.48      0.59      1186

    accuracy                           0.86      5834
   macro avg       0.82      0.72      0.75      5834
weighted avg       0.86      0.86      0.85      5834



In [5]:
# Genetic Algorithm
params_ga = tun.Tuning(n_trials=trials, sampler = optuna.samplers.NSGAIISampler())
ga_scores = tun.Best_MLP_model(params_ga)
ga_scores = ga_scores.rename(columns={'Tree-structured Parzen Estimator': 'Genetic Algorithm'})

[I 2023-03-02 08:49:14,886] A new study created in memory with name: no-name-88944375-24f2-40f2-afc7-6d91ba362432
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 08:50:03,910] Trial 0 finished with value: 0.855286507100956 and parameters: {'units': 37, 'alpha': 0.0639855974134986, 'dropout': 0.2706011729300077, 'lr': 9.687543316938774e-05, 'batch_size': 256, 'l2_kern': 1.3148403748029467e-05, 'l2_bias': 3.4149237575958514e-05}. Best is trial 0 with value: 0.855286507100956.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 08:50:20,519] Trial 1 finished with value: 0.8548250812999321 and parameters: {'units': 8, 'alpha': 0.08597228454486455, 'dropout': 0.2668954759197085, 'lr': 0.0010543975833539796, 'batch_size': 256, 'l2_kern': 1.3831552282488648e-05, 'l2_bias': 0.002035155876841646}. Best is trial 0 with value: 0.855286507100956.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


...

195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 10:00:37,147] Trial 99 finished with value: 0.8507171995067256 and parameters: {'units': 26, 'alpha': 0.01582369865033944, 'dropout': 0.36605278975259387, 'lr': 0.0068498967394983725, 'batch_size': 32, 'l2_kern': 0.0008278751526815582, 'l2_bias': 0.008805583875045634}. Best is trial 57 with value: 0.855634275696429.


Number of finished trials: 100
Best trial:
  Value: 0.855634275696429
    units: 10
    alpha: 0.0369263263673357
    dropout: 0.2245926031678811
    lr: 0.0004138360582475948
    batch_size: 256
    l2_kern: 0.000743392789060623
    l2_bias: 0.0025376081364391436


C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:111: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8552873220618412 while test mean AUC score is 0.860078502099028.
183/183 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

          No       0.88      0.96      0.92      4648
         Yes       0.77      0.49      0.60      1186

    accuracy                           0.87      5834
   macro avg       0.82      0.73      0.76      5834
weighted avg       0.86      0.87      0.85      5834



In [6]:
# Quasi Monte Carlo
params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
mc_scores = tun.Best_MLP_model(params_mc)
mc_scores = mc_scores.rename(columns={'Tree-structured Parzen Estimator': 'Quasi Monte Carlo'})

C:\Users\User\AppData\Local\Temp\ipykernel_10400\530249616.py:2: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  params_mc = tun.Tuning(n_trials=trials, sampler = optuna.samplers.QMCSampler())
[I 2023-03-02 10:35:06,186] A new study created in memory with name: no-name-f14cb04a-39ec-4c62-8c4b-c7e865872c5a
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))


195/195 [==============================] - 0s 2ms/step


[I 2023-03-02 10:36:04,047] Trial 0 finished with value: 0.8549736808347589 and parameters: {'units': 30, 'alpha': 0.056201901691126946, 'dropout': 0.22439426644574495, 'lr': 0.00010255328295486499, 'batch_size': 64, 'l2_kern': 0.0003012039820865274, 'l2_bias': 8.911974421834062e-05}. Best is trial 0 with value: 0.8549736808347589.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-02 10:36:04,101] The parameter 'batch_size' in trial#1 is sampled independently by using `RandomSampler` instead of `QMCSampler` (optimization performance may be degraded). `QMCSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling`

195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 10:37:18,335] Trial 1 finished with value: 0.8248916552520087 and parameters: {'units': 2, 'alpha': 0.01, 'dropout': 0.2, 'lr': 5.000000000000004e-05, 'batch_size': 256, 'l2_kern': 0.0018987461541348789, 'l2_bias': 7.693462686178146e-05}. Best is trial 0 with value: 0.8549736808347589.
C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:37: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=self.create_model, verbose=0))
[W 2023-03-02 10:37:18,336] The parameter 'batch_size' in trial#2 is sampled independently by using `RandomSampler` instead of `QMCSampler` (optimization performance may be degraded). `QMCSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of

...

195/195 [==============================] - 0s 1ms/step


[I 2023-03-02 11:55:43,966] Trial 99 finished with value: 0.8546936494511114 and parameters: {'units': 32, 'alpha': 0.044453125, 'dropout': 0.24453125, 'lr': 8.563836422710964e-05, 'batch_size': 16, 'l2_kern': 5.935229272296989e-05, 'l2_bias': 0.00012634629176544695}. Best is trial 13 with value: 0.8558005150328728.


Number of finished trials: 100
Best trial:
  Value: 0.8558005150328728
    units: 14
    alpha: 0.026875000000000003
    dropout: 0.29375
    lr: 0.0009846896585947416
    batch_size: 512
    l2_kern: 0.0064938163157621165
    l2_bias: 0.00020535250264571456


C:\Users\User\AppData\Local\Temp\ipykernel_10400\3785980822.py:111: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('classifier', KerasClassifier(build_fn=tun.create_model, verbose=0))


37/37 [==============================] - 0s 2ms/step
Train mean AUC score is 0.8549182783179976 while test mean AUC score is 0.8636452408777211.
183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          No       0.88      0.97      0.92      4648
         Yes       0.78      0.47      0.58      1186

    accuracy                           0.86      5834
   macro avg       0.83      0.72      0.75      5834
weighted avg       0.86      0.86      0.85      5834



## Optimal parameters comparison

In [9]:
params_all = pd.DataFrame(zip([item[1] for item in params_tpe], [item[1] for item in params_rs], 
                              [item[1] for item in params_ga], [item[1] for item in params_mc]),
                          index =['NEURONS LAYER 1 ',"AlPHA_l", 'DROPOUT LAYER 1 ','LEARNING RATE',"BATCH SIZE",
                                  "KERNEL REGULARIZER_1","BIAS REGURALIZER_1"] ,
                          columns = ['TPE','RS','GA', 'MC'])

params_all

,TPE,RS,GA,MC
NEURONS LAYER 1,30.000000,13.000000,10.000000,14.000000
AlPHA_l,0.021675,0.026858,0.036926,0.026875
DROPOUT LAYER 1,0.436936,0.277678,0.224593,0.293750
LEARNING RATE,0.000143,0.000281,0.000414,0.000985
BATCH SIZE,256.000000,256.000000,256.000000,512.000000
KERNEL REGULARIZER_1,0.006809,0.000802,0.000743,0.006494
BIAS REGURALIZER_1,0.000258,0.000777,0.002538,0.000205


## Metrics Summary

In [10]:
all_scores = pd.concat([tpe_scores, rs_scores, ga_scores, mc_scores], axis =1)
all_scores

,Tree-structured Parzen Estimator,Random Sampler,Genetic Algorithm,Quasi Monte Carlo
Accuracy,0.864930,0.864415,0.866472,0.864415
Precision,0.774862,0.764391,0.767411,0.775453
Recall,0.473019,0.481450,0.492411,0.468803
F1-score,0.587435,0.590792,0.599897,0.584341
AUC,0.718975,0.721792,0.727165,0.717082
H-measure,0.334284,0.337359,0.347785,0.330960


## Save results

In [11]:
# Save the clean dataset

all_scores.to_csv(r'C:\Users\.....\credit_risk_dataset_1_Layer_scores.csv', index=False)

params_all.to_csv(r'C:\Users\.......\credit_risk_dataset_1_Layer_structure.csv', index=False)
